In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Smote**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image
from imblearn.over_sampling import SMOTE
import collections

# Setup Google Drive path
base_path = "/content/gdrive/MyDrive/Classes"

# Load images and convert to a suitable numeric format
def load_images(data_path):
    images = []
    labels = []
    label_names = os.listdir(data_path)  # list directories for each class
    label_dict = {name: i for i, name in enumerate(label_names)}  # Map labels to integer indices
    for label in label_names:
        class_folder = os.path.join(data_path, label)
        for filename in os.listdir(class_folder):
            filepath = os.path.join(class_folder, filename)
            img = load_img(filepath, target_size=(128, 128))  # Resize images
            img_array = img_to_array(img)
            images.append(img_array.flatten())
            labels.append(label_dict[label])  # Use integer index for labels
    return np.array(images), np.array(labels), label_names

images, labels, label_names = load_images(base_path)

# Preprocessing: flatten images and normalize pixel values
X_train = images.reshape(-1, 128*128*3) / 255.0  # Normalize and flatten images

# Determine the sampling strategy dynamically
original_distribution = collections.Counter(labels)
sampling_strategy = {label_idx: 600 for label_idx, count in original_distribution.items() if count < 600}

# Define SMOTE with the specific strategy
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)

# Apply SMOTE
X_resampled, y_resampled = smote.fit_resample(X_train, labels)
X_resampled_reshaped = X_resampled.reshape(-1, 128, 128, 3)

# Function to save synthetic images
def save_images(images, labels, label_names, save_dir="/content/gdrive/MyDrive/Classes_SMOTE2"):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for idx, img_array in enumerate(images):
        label_index = labels[idx]
        label_folder = os.path.join(save_dir, label_names[label_index])
        if not os.path.exists(label_folder):
            os.makedirs(label_folder)
        img = Image.fromarray((img_array * 255).astype(np.uint8))
        img.save(os.path.join(label_folder, f"smote_{idx}.jpg"))

# Save synthetic images
save_images(X_resampled_reshaped, y_resampled, label_names, "/content/gdrive/MyDrive/Classes_SMOTE2")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Count of images**

In [ ]:
base_path = '/content/gdrive/MyDrive/Classes_SMOTE2'
import os

def count_images_by_folder(base_path):
    folder_counts = {}
    # Check each folder in the base directory
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):  # Make sure it's a directory
            image_files = [file for file in os.listdir(folder_path)
                           if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'))]
            folder_counts[folder] = len(image_files)
    return folder_counts

# Run the function and print the results
image_counts = count_images_by_folder(base_path)
for folder, count in image_counts.items():
    print(f"There are {count} image files in the '{folder}' folder.")


There are 600 image files in the 'ROI_chickenpox' folder.
There are 871 image files in the 'ROI_psoriasis' folder.
There are 600 image files in the 'ROI_monkeypox' folder.
There are 600 image files in the 'ROI_measles' folder.
There are 600 image files in the 'ROI_hives' folder.
There are 645 image files in the 'ROI_eczema' folder.
There are 600 image files in the 'ROI_dermatitis' folder.
There are 600 image files in the 'ROI_bullous' folder.
There are 637 image files in the 'ROI_acne' folder.
There are 600 image files in the 'Normal' folder.
